In [39]:
library(plyr)
library(dplyr)
library(reshape2)

In [40]:
path_to_data = "data/Callejero Plano geometrico Tomas Lopez/"

In [41]:
csv_files = list.files(path_to_data, pattern = "^.*\\.csv$")

In [105]:
for(f in seq_along(csv_files)){
    full_path = paste0(path_to_data, csv_files[f])
    
    d = read.csv(full_path, skip = 3) 
    d_names = d %>%
    select(contains(c("letra", "nº")))%>%
    names
    l_coord = length(d_names)/2
    for(n in 1:l_coord){
        vector_names = d_names[c(n, n+l_coord)]
        dd = d[,vector_names]%>%
            apply(FUN = paste0, MARGIN = 1, collapse= "")
        dd_clean = gsub(" ", "", dd)
        d = cbind(d, dd_clean)
        names(d)[length(names(d))] = paste0("coord", n)
    }
    if("manzana"%in% names(d)){
        df = d %>%
            select(1, "manzana", contains("coord")) %>%
            melt(id.var=c(1, 2), value.name="coord")%>%
            mutate(value = gsub(" ", "", coord))%>%
            filter(!(value%in%c("NA", "NANA")))%>%
            select(-variable)
    }
    else{
        df = d %>%
            select(1, contains("coord")) %>%
            melt(id.var=c(1), value.name="coord")%>%
            mutate(value = gsub(" ", "", coord))%>%
            filter(!(value%in%c("NA", "NANA")))%>%
            select(-variable)
    }
    write.csv(df, paste0("data/callejero_plano_long_table/long_table_",csv_files[f]), row.names = FALSE)
    
}